In [1]:
import itertools
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra #フィボナッチヒープを使用している
import math
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import time
from InfMaxProblem import ICmodel
import sys
sys.setrecursionlimit(20000)

%matplotlib inline

In [2]:
def make_random_graph_list(V_size, E_size):
    network_list = np.array([np.random.choice(range(V_size),2,replace=False) for i in range(E_size)])
    network_list = np.unique(network_list, axis=0)
    weighted = np.random.uniform(0,1,len(network_list))
    
    # from to と　weightedを連結
    return np.vstack([network_list.T, weighted]).T

In [223]:
# 確率を-log(p)変換する
def neg_log(network_np):
    return np.vstack([network_np.T[:2], -np.log(network_np.T[2])]).T

In [218]:
network_np = make_random_graph_list(5000, 10000)

In [219]:
network_np_log = neg_log(network_np)

In [220]:
# 空の有向グラフを作成
G = nx.DiGraph()

In [221]:
# 重み付きの枝を加える
G.add_weighted_edges_from(network_np_log)

In [207]:
# plt.figure(figsize=(15,5))
# nx.draw_networkx(G)

In [208]:
def compute_MIP_G(G, theta):
    return dict(nx.all_pairs_dijkstra(G, cutoff=-np.log(theta)))

In [209]:
def compute_MIOA(v, G_table):
    MIOA = nx.DiGraph()
    for u_pass in G_table[v][1].values():
        nx.add_path(MIOA, u_pass)
    return MIOA

# theta = 0.00001
# G_table = compute_MIP_G(G, theta)

# MIOA = compute_MIOA(0, G_table)
# # nx.draw_networkx(MIOA)

In [210]:
def compute_MIIA(v, G_table):
    MIIA = nx.DiGraph()
    for node in G:
        if v in G_table[node][1]:
            nx.add_path(MIIA, G_table[node][1][v])
    return MIIA

# theta = 0.00001
# G_table = compute_MIP_G(G, theta)

# MIIA = compute_MIIA(2, G_table)
# # nx.draw_networkx(MIIA)

In [211]:
def compute_ap(ap, S, MIIA_v, v, G_table):
    # MIIAtreeの末端から計算すれば、効率的になるため、ソート
    from_u_to_v_pass_len = dict(nx.single_target_shortest_path_length(MIIA_v, v))
    sorted_pass_len = sorted(from_u_to_v_pass_len.items(), key=lambda x:x[1], reverse=True)
    
    for u, _ in sorted_pass_len:
        if u in S:
            ap[(u, MIIA_v)] = 1
        else:
            N_in = MIIA_v.in_edges([u])
            if not N_in:
                ap[(u, MIIA_v)] = 0
            else:
                prod = 1
                for w, _ in N_in:
                    prod *= 1 - ap[(w, MIIA_v)] * np.exp(-G_table[w][0][u])
                ap[(u, MIIA_v)] = 1 - prod
                
# theta = 0.0001
# v = 2
# S = [3]
# G_table = compute_MIP_G(G, theta)
# MIIA_v = compute_MIIA(v, G_table)
# # nx.draw_networkx(MIIA_v)
# ap = dict()
# compute_ap(ap, S, MIIA_v, v, G_table)
# ap

In [212]:
def compute_alpha(alpha, v, MIIA_v, S, G_table, ap):
    # 木のルートからはじめると効率的に計算できる
    from_u_to_v_pass_len = dict(nx.single_target_shortest_path_length(MIIA_v, v))
    sorted_pass_len = sorted(from_u_to_v_pass_len.items(), key=lambda x:x[1])

    for u, _ in sorted_pass_len:
        if u == v:
            alpha[(MIIA_v, u)] = 1
        else:
            w = list(MIIA_v.out_edges([u]))[0][1]
            if w in S:
                alpha[(MIIA_v, u)] = 0
            else:
                N_in = MIIA_v.in_edges([w])
                prod = 1
                for u_, _ in N_in:
                    if u_ != u:
                        prod *= 1 - ap[(u_, MIIA_v)] * np.exp(-G_table[u_][0][w])
                alpha[(MIIA_v, u)] = alpha[(MIIA_v, w)] * np.exp(-G_table[u][0][w]) * prod

# theta = 0.0001
# v = 2
# S = [3]
# G_table = compute_MIP_G(G, theta)
# MIIA_v = compute_MIIA(v, G_table)
# nx.draw_networkx(MIIA_v)
# ap = dict()
# compute_ap(ap, S, MIIA_v, v, G_table)

# alpha = dict()
# compute_alpha(alpha, v, MIIA_v, S, G_table, ap)
# alpha

In [224]:
def compute_MIA(G, k, theta):
    # initialization
    # 2
    S = []
    
    # 3
    IncInf = dict(zip(G.nodes(), [0]*len(G)))
    
    MIIA = dict()
    MIOA = dict()
    ap = dict()
    alpha = dict()
    
    # ダイクストラの計算を一括で行う
    G_table = compute_MIP_G(G, theta)
    
    
    # 4
    for v in G:
        # 5
        MIIA[v] = compute_MIIA(v, G_table)
        MIOA[v] = compute_MIOA(v, G_table)
        
        # 6
        for u in MIIA[v]:
            ap[(u, MIIA[v])] = 0
        
        # 7
        compute_alpha(alpha, v, MIIA[v], S, G_table, ap)
        
        # 8
        for u in MIIA[v]:
            IncInf[u] += alpha[(MIIA[v], u)] * (1 - ap[(u, MIIA[v])])

    # main loop
    # 13
    for i in range(k):
        # 14
        u = sorted(IncInf.items(), key=lambda x:x[1], reverse=True)[0][0]
        print("u",u)
        
        # 16
        for v in MIOA[u]:
            if not v in S:
                for w in MIIA[v]:
                    if not w in S:
                        IncInf[w] -= alpha[(MIIA[v], w)] * (1 - ap[(w, MIIA[v])])
        # 22
        S += [u]
        
        if len(S) == k:
            return S
        
        # 23
        for v in MIOA[u]:
            if not v in S:
                # 24
                compute_ap(ap, S, MIIA[v], v, G_table)
                
                # 25
                compute_alpha(alpha, v, MIIA[v], S, G_table, ap)
                
                # 27
                for w in MIIA[v]:
                    if not w in S:
                        # 28
                        IncInf[w] += alpha[(MIIA[v], w)] * (1 - ap[(w, MIIA[v])])

# ランダムネットワーク行う場合

In [232]:
#network_np = make_random_graph_list(500, 1000)

# データを読み込んで行う場合

In [233]:
# データの読み込み
# 枝確率を計算済みのネットワークを読み込む
network = pd.read_csv("data.csv")
network.head()

,# FromNodeId,ToNodeId,p
0,0,4,0.008000
1,0,5,0.005682
2,0,7,0.033333
3,0,8,0.009615
4,0,9,0.066667


In [234]:
# numpy型に変換
network_np = network.values

In [235]:
network_np_log = neg_log(network_np)

In [236]:
# 空の有向グラフを作成
G = nx.DiGraph()

In [237]:
# 重み付きの枝を加える
G.add_weighted_edges_from(network_np_log)

In [ ]:
%time compute_MIA(G, 3, 0.1)